In [ ]:
using QuantumCollocation
using NamedTrajectories

using LinearAlgebra
using CairoMakie

In [ ]:
H_drift = zeros(2, 2)
H_drives = [GATES[:X], GATES[:Y]]
U_goal = GATES[:X]
T = 50
Δt = .2

probs = Dict()

probs["default"] = UnitarySmoothPulseProblem(
    H_drift,
    H_drives,
    U_goal,
    T,
    Δt;
    free_time=true,
    timesteps_all_equal=false,
    hessian_approximation=true,
)

solve!(probs["default"]; max_iter=25)


In [ ]:
plot(probs["default"].trajectory)

# Branch `offset_l2`
-----

## Changes to `problem_templates.jl`

We modified the problem template to track the control provided by `a_track`.

The regularizer you pass to `R_a` will determine the weight of this value.

```Julia
	J += QuadraticRegularizer(:a, traj, R_a; values=a_track)
```

I am keeping `a_guess` separate, so you can randomly initialize even if you want to eventually track a certain control.

## Changes to `objectives.jl`

The key changes is the values argument, where we plug in `a_track`.

```Julia
function QuadraticRegularizer(;
	name::Symbol=nothing,
	times::AbstractVector{Int}=1:traj.T,
	dim::Int=nothing,
	R::AbstractVector{<:Real}=ones(traj.dims[name]),
	values::Union{Nothing,AbstractArray{<:Real}}=nothing,
	eval_hessian=true,
	timestep_symbol=:Δt
)
```

# Visual test
-----

Compare tracking (you should match the original picture) to no tracking (the pictures will be different).

In [ ]:
# Weight the objective to make the controls match the previous solution
probs["track"] = UnitarySmoothPulseProblem(
    H_drift,
    H_drives,
    U_goal,
    T,
    Δt;
    a_track=probs["default"].trajectory[:a],
    free_time=true,
    timesteps_all_equal=false,
    hessian_approximation=true,
    R_a=100.,
    R_dda=100.
)

solve!(probs["track"]; max_iter=100)

In [ ]:
plot(probs["track"].trajectory)

In [ ]:
# Weight the objective to make the controls match the previous solution
probs["no-track"] = UnitarySmoothPulseProblem(
    H_drift,
    H_drives,
    U_goal,
    T,
    Δt;
    free_time=true,
    timesteps_all_equal=false,
    hessian_approximation=true,
    R_a=100.,
    R_dda=100.
)

solve!(probs["no-track"]; max_iter=100)

In [ ]:
plot(probs["no-track"].trajectory)